## BETTER WAY 22 - 딕셔너리와 튜플보다는 헬퍼 클래스로 관리하자
## Item22 - Prefer Helper Classes Over Bookkeeping with Dictionaries and Tuples

파이썬의 딕셔너리 타입은 객체의 수명이 지속되는 동안 동적인 내부 상태를 관리하는 용도로 좋다. '동적'이란 예상치 못한 식별자(identifier)들을 관리해야 하는 상황을 말한다. <br />
예를 들어 학생의 성적을 기록하고 싶을 경우, 학생별로 딕셔너리에 이름을 저장하는 클래스를 아래와 같이 작성할 수 있다.

In [1]:
class SimpleGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = []
        
    def report_grade(self, name, score):
        self._grades[name].append(score)
        
    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

In [2]:
book = SimpleGradebook()
book.add_student('Isaac Newton')
book.report_grade('Isaac Newton', 90)
book.report_grade('Isaac Newton', 80)
book.report_grade('Isaac Newton', 80)

print(book.average_grade('Isaac Newton'))

83.33333333333333


이번엔 `SimpleGradebook` 클래스를 확장해서 과목별로 성적을 저장할 경우 아래와 같이 작성할 수 있다.

In [3]:
class BySubjectGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
    
    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append(grade)
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        print(by_subject)
        return total / count

In [4]:
book = BySubjectGradebook()
book.add_student('최종현')
book.report_grade('최종현', '수학', 75)
book.report_grade('최종현', '수학', 65)
book.report_grade('최종현', '체육', 90)
book.report_grade('최종현', '체육', 95)

print(book.average_grade('최종현'))

{'체육': [90, 95], '수학': [75, 65]}
81.25


이제는 수업의 최종 성적에서 각 점수가 차지하는 비중르 매겨서 중간고사와 기말고사를 쪽지시험 보다 중요하게 만들려고 한다. 이 기능을 구현하는 방법 중 하나는 가장 안쪽 딕셔너리를 변경해서 과목(키)를 성적(값)에 매핑하지 않고, 성적과 비중을 담은 튜플 `(score, weight)`에 매핑하는 방법이다.

In [5]:
class WeightedGradebook(object):
    def __init__(self):
        self._grades = {}
        
    def add_student(self, name):
        self._grades[name] = {}
        
    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject.setdefault(subject, [])
        grade_list.append((score, weight))
        
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0
        print(by_subject)
        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0
            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight
            score_sum += subject_avg / total_weight
            score_count += 1
        return score_sum / score_count

In [6]:
book = WeightedGradebook()
book.add_student('Albert Einstein')
book.report_grade('Albert Einstein', 'Math', 80, 0.10)
book.report_grade('Albert Einstein', 'Math', 80, 0.10)
book.report_grade('Albert Einstein', 'Math', 70, 0.80)
book.report_grade('Albert Einstein', 'Gym', 100, 0.40)
book.report_grade('Albert Einstein', 'Gym', 85, 0.60)
print(book.average_grade('Albert Einstein'))

{'Gym': [(100, 0.4), (85, 0.6)], 'Math': [(80, 0.1), (80, 0.1), (70, 0.8)]}
81.5


위의 예제들은 갈 수록 복잡해진다. 바로 위의 예제에서는 위치 인수가 많아지므로 각각의 인수들이 무엇을 의미하는지 명확해지지 않는다.

## 클래스 리팩토링

관리하기가 복잡한 클래스일 경우 클래스를 세분화(?)하는 것이 좋다. 아래의 코드는 성적들을 기록한 단일 과목을 표현한 클래스(`Subject`)와 학생이 공부한 과목들을 표현하는 클래스(`Student`), 마지막으로 학생의 이름을 담는 클래스(`Gradebook`)이렇게 세 가지 클래스로 나눈 예제이다. 여기서는 성적을 담는 변수인 `Grade`를 `collections`모듈의 `namedtuple`을 이용했다. `namedtuple`에 대한 자세한 설명은 [collections 모듈 - namedtuple](http://excelsior-cjh.tistory.com/entry/collections-%EB%AA%A8%EB%93%88-namedtuple?category=966334) 에서 참고할 수 있다.

In [7]:
import collections

Grade = collections.namedtuple('Grade', ('score', 'weight'))

class Subject(object):
    def __init__(self):
        self._grades = []
        
    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))
        
    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight

In [8]:
class Student(object):
    def __init__(self):
        self._subjects = {}

    def subject(self, name):
        if name not in self._subjects:
            self._subjects[name] = Subject()
        return self._subjects[name]

    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count

In [9]:
class Gradebook(object):
    def __init__(self):
        self._students = {}

    def student(self, name):
        if name not in self._students:
            self._students[name] = Student()
        return self._students[name]

In [10]:
book = Gradebook()
albert = book.student('Albert Einstein')
math = albert.subject('Math')
math.report_grade(80, 0.10)
math.report_grade(80, 0.10)
math.report_grade(70, 0.80)
gym = albert.subject('Gym')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade())

81.5


### 정리
- 다른 딕셔너리나 긴 튜플을 값으로 담은 딕셔너리를 생성하지 않는 것이 좋다
- 내부 상태를 관리하는 딕셔너리가 복잡해지면 여러 헬퍼 클래스를 사용하는 방식으로 관리 코드를 바꾸자.